In [2]:
import os

import cv2
from dask import delayed
import numpy as np
from matplotlib import pyplot as plt
import sklearn
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC


##  Integral Image

Computes rectangle integrals of an image for fast computation of Harr Filters.

In [3]:
def compute_intg_image(img):
    """
    Computes matrix for quick computation of integrals.
    """
    # Create a matrix of zeros with the same dimensions as the input image
    intg_img = np.zeros(img.shape, dtype=np.int64)
    
    # Iterate over each pixel in the input image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Calculate the sum of all pixels above and to the left of the current pixel
            if i > 0:
                intg_img[i, j] += intg_img[i-1, j]
            if j > 0:
                intg_img[i, j] += intg_img[i, j-1]
            if i > 0 and j > 0:
                intg_img[i, j] -= intg_img[i-1, j-1]
            
            # Add the value of the current pixel to the integral image
            intg_img[i, j] += img[i, j]
    
    # Return the computed integral image
    return intg_img

def get_rect(intg_img, si, sj, ei, ej):
    # Calculate the sum of all pixels within the specified rectangular region
    result = intg_img[ei, ej]
    if si > 0:
        result -= intg_img[si-1, ej]
    if sj > 0:
        result -= intg_img[ei, sj-1]
    if si > 0 and sj > 0:
        result += intg_img[si-1, sj-1]
    
    # Return the computed sum of pixels
    return result

In [4]:
def load_folder(folder):
    imgs = []
    for file in os.listdir(folder):
        if ".pgm" in file:
            imgs.append(cv2.imread(folder+"/"+file,-1))
    return imgs

In [5]:
def compute_haar_features(intg_img, haar_filters, feature_size):
    haar_features = []
    
    for filter in haar_filters:
        filter_feature = []
        for i in range(intg_img.shape[0]-filter[0]+1):
            for j in range(intg_img.shape[1]-filter[1]+1):
                if filter[2] == 2:
                    result = ( -get_rect(intg_img, i, j, i+filter[0]-1, j+filter[1]//3-1) 
                              + get_rect(intg_img, i, j+filter[1]//3, i+filter[0]-1, j+2*filter[1]//3-1)*2 
                    - get_rect(intg_img, i, j+2*filter[1]//3, i+filter[0]-1, j+filter[1]-1) )*3/(2*(filter[0])*filter[1])
                if filter[2] == 1:
                    result = ( get_rect(intg_img, i, j, i+filter[0]-1, j+filter[1]//2-1) 
                    - get_rect(intg_img, i, j+filter[1]//2, i+filter[0]-1, j+filter[1]-1) )/((filter[0])*filter[1])*2
                elif filter[2] == 0:
                    result = ( get_rect(intg_img, i, j, i+filter[0]//2-1, j +filter[1]-1) 
                    - get_rect(intg_img, i+filter[0]//2, j, i+filter[0]-1, j+filter[1]-1) )/((filter[0])*filter[1])*2
                filter_feature.append(result)
        haar_features.append(filter_feature)

    return haar_features
def generate_haar_filters(detector_size=19):
    haar_feature_size = 0
    # (r, c, is_vert)
    haar_filters= []
    for i in range(1, detector_size+1):
        for j in range(1, detector_size+1):
            if i % 2 == 0:
                haar_filters.append((i,j, 0))
                haar_feature_size += (19-i+1) * (19-j+1)
            if j % 2 == 0:
                haar_filters.append((i,j, 1))
                haar_feature_size += (19-i+1) * (19-j+1)
            if j % 3 == 0:
                haar_filters.append((i,j, 2))
                haar_feature_size += (19-i+1) * (19-j+1)
    return haar_feature_size, haar_filters

def get_haar_feature_extractor(detector_size):
    haar_feature_size, haar_filters = generate_haar_filters(detector_size)
    @delayed
    def get_haar(intg_img):
        return compute_haar_features(intg_img, haar_filters, haar_feature_size)
    return get_haar

haar_feature_extractor = get_haar_feature_extractor(19)


In [6]:
import tqdm

In [7]:
rand_ind = np.random.permutation(2000)[:1000]
train_x_true = load_folder("train/face")
train_x_true = [train_x_true[i] for i in rand_ind]
train_x_false = load_folder("train/non-face")
train_x_false = [train_x_false[i] for i in rand_ind]
train_y_true = [1] * len(train_x_true)
train_y_false = [0] * len(train_x_false)

train_x = train_x_true + train_x_false
train_y = train_y_true + train_y_false

train_x = delayed(haar_feature_extractor(compute_intg_image(img)) for img in tqdm.tqdm(train_x) )

train_x = train_x.compute(scheduler='single-threaded')
train_y = np.array(train_y)

100%|██████████| 2000/2000 [00:01<00:00, 1300.96it/s]


In [166]:
def Adaboost(train_x, train_y, d=1, T=100):
    face_count = 0
    non_face_count = 0
    for i, label in enumerate(train_y):
        if label:
            face_count += 1
        else:
            non_face_count += 1
    print(face_count, non_face_count)
    w = np.array([1/face_count if train_y[i] else 1/non_face_count for i in range(len(train_y))])

    classifiers = []
    feature_sel = []
    betas = []
    model_threshold = 0
    def select_features(x_data, x_sel):
        return [np.hstack([x[f] for f in x_sel]) for x in x_data]
    predictions = np.zeros(train_y.shape)
    pbar = tqdm.tqdm(range(T))
    for t in pbar:
        w = w / np.sum(w)
        while True: 
            model = RidgeClassifier()
            m_sel = np.random.permutation(len(train_x[0]))[:d]
            m_train_x = select_features(train_x, m_sel)
            model.fit(m_train_x, train_y, sample_weight=w)
            e = np.abs(train_y - model.predict(m_train_x))
            
            beta = np.sum(w * e)
            if beta < 0.5:
                
                feature_sel.append(m_sel)
                w = w * np.power(beta, 1-e)
                classifiers.append(model)
                betas.append(beta+1e-6)
                break
        alpha = -np.log(betas)
        predictions = predictions + alpha[-1]*np.array(classifiers[-1].predict(select_features(train_x, feature_sel[-1])))
        model_threshold = np.min(predictions[train_y==1])
        miss_classification = np.sum((predictions >= model_threshold) != train_y)
        pbar.set_description(f'Error: {miss_classification}')
        if miss_classification == 0:
            break  
        
    
    def model(x):
        preds = np.sum( np.array([c.predict(select_features(x, f_sel)) for c, f_sel in zip(classifiers, feature_sel)]) * alpha.reshape(-1,1), axis=0)
        return preds >= model_threshold
    return model
    

In [167]:
def train_classifier(train_x, train_y):
    final_cascade = []
    x = train_x
    y = np.array(train_y)
    d_size = 1
    while True:
        model=Adaboost(x,y,  200,20)
        y_pred = model(x)
        
        pos_sample = np.argwhere(y_pred==True).flatten()
        new_x = [x[i] for i in pos_sample]
        new_y = [y[i] for i in pos_sample]
        d_size += 1
        final_cascade.append(model)
        if( (y_pred==y).all() ):
            break

        x = new_x
        y = np.array(new_y)
    return final_cascade

In [171]:
cascade = train_classifier(train_x, train_y)

50 200


Error: 0:   0%|          | 0/20 [00:01<?, ?it/s]


In [157]:
cascade

[<function __main__.Adaboost.<locals>.model(x)>]

In [81]:
test_x_true = load_folder("train/face")[600:620]
test_x_false = load_folder("train/non-face")[600:620]
test_y_true = [1] * len(test_x_true)
test_y_false = [0] * len(test_x_false)

test_x = test_x_true + test_x_false
test_y = test_y_true + test_y_false

test_x = delayed(haar_feature_extractor(compute_intg_image(img)) for img in test_x)
test_x = test_x.compute(scheduler='single-threaded')
test_y = np.array(test_y)



In [82]:
def raw_img(cascade, img):
    for model in cascade:
        if not model([img])[0]:
            return False
    return True

In [83]:
(np.array([raw_img(cascade, img) for img in test_x])) == test_y

array([ True,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True, False, False,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False,  True, False,  True])

In [84]:
def check_img(cascade, img):
    intg_img = compute_intg_image(img)
    
    haar_features = [haar_feature_extractor(intg_img).compute()]
    
    for model in cascade:
        if not model(haar_features)[0]:
            return False
    return True

In [86]:
check_img(cascade,load_folder("data")[1])

False

In [191]:
load_folder("data")[0].shape

(19, 19)

In [172]:

  
# define a video capture object
vid = cv2.VideoCapture(0)

while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    img = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), (200,200))
    cimg = img.copy()
    k_size = 80
    for i in range(0,img.shape[0]-k_size+1, k_size//2):
        for j in range(0,img.shape[1]-k_size+1, k_size//2):
            if(check_img(cascade, cv2.resize(img[i:i+k_size, j:j+k_size], (19,19))) ):
                cv2.rectangle(cimg, (j,i), (j+k_size, i+k_size), 255, 2)
                cv2.imshow('face',cv2.resize(img[i:i+k_size, j:j+k_size], (50,50)) )
                cv2.waitKey(1)
            cv2.imshow('frame', cimg)
            cv2.waitKey(5)
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
def get_rect(si, sj, ei, ej):
    return Symbol(f"intg_img[{str(ei+1)}, {str(ej+1)}]") - Symbol(f"intg_img[{str(si)}, {str(ej+1)}]")  - Symbol(f"intg_img[{str(ei+1)}, {str(sj)}]") + Symbol(f"intg_img[{str(si)}, {str(sj)}]")
i = Symbol('i')
j = Symbol('j')
h = Symbol('h')
w = Symbol('w')
str( get_rect(i, j, i + h/3 -1, j+w-1 )/(2/3) - get_rect(i+h/3, j, i + 2*h//3-1, j+ w -1 )/(1/3) - get_rect(i+2 * h//3, j, i + h-1, j+w-1 )/ (2/3))

                                                                  